In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from sklearn.metrics import roc_curve, roc_auc_score,confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,GridSearchCV
from optbinning import OptimalBinning
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm    


In [ ]:

d= pd.read_excel(r'doctype.xlsx')
a= pd.read_csv(r'C:\Users\GauravKumar\Downloads\report_dumps_MIS Credit Strategic_ADHOC_000000000000_PSI.csv', dtype={"loanAccountNumber":"string"})
b= pd.read_csv(r'C:\Users\GauravKumar\Downloads\report_dumps_MIS Credit Strategic_ADHOC_000000000001_PSI.csv', dtype={"loanAccountNumber":"string"})
c= pd.read_csv(r'C:\Users\GauravKumar\Downloads\report_dumps_MIS Credit Strategic_ADHOC_000000000002_PSI.csv', dtype={"loanAccountNumber":"string"})
df=a.append(b)
df=df.append(c)

df=df[df['loanAccountNumber'].notna()]
e=pd.read_excel('credoscore.xlsx', dtype={"loanAccountNumber":"string"})
df=df.set_index('loanAccountNumber')
e=e.set_index('loanAccountNumber')

df=pd.concat([df,e],join='inner', axis=1)  ### Add d if want Advance AI
df=df.reset_index()
df=pd.merge(df,d,on='docType',how='left')

df['ObsFSTPD30']=np.where(((df['ObsFPD30']==1) | (df['ObsSPD30']==1) | (df['ObsTPD30']==1)),1,0)
df['DefFSTPD30']=np.where(((df['DefFPD30']==1) | (df['DefSPD30']==1) | (df['DefTPD30']==1)),1,0)
df=df[(df['ObsFSTPD30']==1)]
df= df[(df['disbursement_date']>=20220101) & (df['disbursement_date']<20221101)]
print(df['DefFSTPD30'].sum()/df.shape[0])

df1=df.copy()

df1.reset_index(inplace=True)

In [ ]:
################################# android_model_15 ########################################
df1['Credo:1']= np.where((df1['android_model_41_score'] <=456),1,0)
df1['Credo:2']= np.where((df1['android_model_41_score'] >=456) &  (df1['android_model_41_score'] < 480),1,0)
df1['Credo:3']= np.where((df1['android_model_41_score'] >=480) &  (df1['android_model_41_score'] < 500),1,0)
df1['Credo:4']= np.where((df1['android_model_41_score'] >=500) &  (df1['android_model_41_score'] < 524),1,0)
df1['Credo:5']= np.where((df1['android_model_41_score'] >=524),1,0)
df1['Credo:0']= np.where(df1['android_model_41_score'].isnull(),1,0)

################################# tsaOpenedMinutes ########################################
df1['tsaOpenedMinutes:1']= np.where((df1['tsaOpenedMinutes'] <3),1,0)
df1['tsaOpenedMinutes:2']= np.where((df1['tsaOpenedMinutes'] >=3) & (df1['tsaOpenedMinutes'] <5),1,0)
df1['tsaOpenedMinutes:3']= np.where((df1['tsaOpenedMinutes'] >=5) & (df1['tsaOpenedMinutes'] < 91),1,0)
df1['tsaOpenedMinutes:4']= np.where((df1['tsaOpenedMinutes'] >=91) & (df1['tsaOpenedMinutes'] < 287323),1,0)
df1['tsaOpenedMinutes:0']= np.where((df1['tsaOpenedMinutes'] >=287323),1,0)
################################# IndustrySubindustry ########################################

df1['industrysubindustry:BIN1']=np.where(df1['industrysubindustry'].isin(['TECHNOLOGY - TECHNOLOGY','TECHNOLOGY - SERVICES','SERVICES - TECHNOLOGY'
 ,'SERVICES - MANUFACTURING','SERVICES - MEDIA / NEWS'
 ,'SERVICES - HOTEL / RESORT / LODGE'
 ,'TECHNOLOGY - START UP COMPANY (FINTECH / OTHERS)'
 ,'MANUFACTURING - AUTO ANCILLARY'
 ,'SERVICES - EDUCATION / SCHOOL / COLLEGE / ONLINE COACHING'
 ,'AGRICULTURE - AGRICULTURE /  HORTICULTURE','SERVICES - AVIATION'
 ,'MANUFACTURING - FMCG'
 ,'MANUFACTURING - BUILDING AND CONSTRUCTION (E.G. CEMENT, STEEL, PAINT, BRICK, TILES, INTERIOR DECORATION, BATHROOM FITOUTS, ELEVATOR MANUFACTURES, RESIDENTIAL GRADE GLASS MAKERS)'
 ,'FINANCIAL SERVICES - PAWN SHOPS'
 ,'MANUFACTURING - OIL AND GAS / ANCILLARY (RIG INSTALLATION / MAINTENANCE)'
,'SERVICES - MARKETING / ADVERTISEMENT (TV / PAPER / RADIO)']),1,0)

df1['industrysubindustry:BIN2']=np.where(df1['industrysubindustry'].isin(['SERVICES - ENTERTAINMENT (CINEMA HALLS / EVENT MANAGEMENT)'
 ,'MANUFACTURING - CONSTRUCTION'
 ,'GOVERNMENT INSTITUTION - GOVERNMENT INSTITUTION'
 ,'FINANCIAL SERVICES - BANKING','FINANCIAL SERVICES - OTHERS'
 ,'SERVICES - OFFICE FURNISHING / FITOUT'
 ,'SERVICES - CO-WORKING SPACE / BUSINESS CENTRE'
 ,'TECHNOLOGY - IT / TECH CONSULTANCY / PROJECTS / AI ML SOLUTION PROVIDERS'
 ,'SERVICES - HOSPITAL','TRADING - OTHERS']),1,0)

df1['industrysubindustry:BIN3']=np.where(df1['industrysubindustry'].isin(['MANUFACTURING - WHITE GOODS'
 ,'SERVICES - REAL ESTATE - COMMERCIAL / RESIDENTIAL'
 ,'SERVICES - LIFE / MEDICAL INSURANCE','MANUFACTURING - PHARMACEUTICAL'
 ,'MANUFACTURING - OTHERS'
 ,'GOVERNMENT INSTITUTION - GOVERNMENT DEPARTMENT / INSTITUTIONS'
 ,'SERVICES - OTHERS','TECHNOLOGY - OTHERS'
 ,'FINANCIAL SERVICES - FINANCIAL SERVICES','MANUFACTURING - MANUFACTURING'
 ,'MANUFACTURING - MEDICAL EQUIPMENT','SERVICES - FITNESS (GYM)'
 ,'FINANCIAL SERVICES - FINANCE COMPANY / CONSUMER FINANCE COMPANY']),1,0)
df1['industrysubindustry:BIN4']=np.where(df1['industrysubindustry'].isin(['SERVICES - CASINO / GAMBLING'
 ,'SERVICES - RESTAURANT / FOODCOURT / CAFE (BRICK AND MORTAR) / PUBS'
 ,'TRADING - COMPUTER / COMPUTER PERIPHERALS'
 ,'AGRICULTURE - FOOD PROCESSING','SERVICES - CALL CENTRE / BPO'
 ,'SERVICES - TELECOM / MOBILE TELEPHONY'
 ,'SERVICES - LOGISTICS / WAREHOUSING / COLD STORAGE'
 ,'SERVICES - DIGITAL PAYMENT / WALLETS','AGRICULTURE - OTHERS'
 ,'GOVERNMENT INSTITUTION - OTHERS','TECHNOLOGY - E-COMMERCE'
 ,'MANUFACTURING - TEXTILE / GARMENT'
 ,'MANUFACTURING - PERSONAL HEALTHCARE/HYGIENE'
 ,'SERVICES - MANPOWER / MANNING AGENCIES (CONSTRUCTION INDUSTRY)']),1,0)
df1['industrysubindustry:BIN0']=np.where(df1['industrysubindustry'].isin(['SERVICES - RETAIL (SUPER MARKET / GROCERY)'
 ,'SERVICES - RESTAURANT (HOME DELIVERY / CLOUD KITCHEN)'
 ,'SERVICES - TRAVEL & TOURISM','MANUFACTURING - PASSENGER 4 WHEELER (CAR)'
 ,'SERVICES - CARGO TRANSPORT','SERVICES - SERVICES'
 ,'SERVICES - RETAIL / MALL / HYPER MARKET'
 ,'SERVICES - TAXI SERVICE; TRANSPORT AGGREGATOR (GRAB, UBER ETC)'
 ,'SERVICES - MANPOWER / MANNING AGENCIES(SECURITY SERVICES; HOUSE KEEPING; LOGISTICS / PORTS)'
 ,'SERVICES - GROOMING (SALON / PARLOUR)','AGRICULTURE - SERVICES'
 ,'TRADING - SERVICES']) | (df1['industrysubindustry'].isnull()),1,0)

################################# loanRequestTenure ########################################
df1['loanRequestTenure:BIN1']=np.where(df1['loanRequestTenure'].isin([6]),1,0)
df1['loanRequestTenure:BIN2']=np.where(df1['loanRequestTenure'].isin([3,9]),1,0)
df1['loanRequestTenure:BIN3']=np.where(df1['loanRequestTenure'].isin([18]),1,0)
df1['loanRequestTenure:BIN4']=np.where(df1['loanRequestTenure'].isin([12]),1,0)
df1['loanRequestTenure:BIN0']=np.where(df1['loanRequestTenure'].isin([24]),1,0)

################################# monthlyIncome ########################################

df1['monthlyIncome:BIN1']= np.where((df1['monthlyIncome'] < 20102),1,0)
df1['monthlyIncome:BIN2']= np.where((df1['monthlyIncome'] >= 20102) &  (df1['monthlyIncome'] <31275),1,0)
df1['monthlyIncome:BIN3']= np.where((df1['monthlyIncome'] >= 31275) &  (df1['monthlyIncome'] <35010),1,0)
df1['monthlyIncome:BIN4']= np.where((df1['monthlyIncome'] >= 35010) &  (df1['monthlyIncome'] <50042),1,0)
df1['monthlyIncome:BIN0']= np.where((df1['monthlyIncome'] >= 50042),1,0)

################################# natureofwork ########################################
df1['natureofwork:BIN1']=np.where(df1['natureofwork'].isin(['OWNER','NEWSCASTER/MEDIA PERSONALITY','ACCOUNTANT','ENGINEER']),1,0)
df1['natureofwork:BIN2']=np.where(df1['natureofwork'].isin(['ARCHITECT/INTERIOR DECORATOR','BANK EMPLOYEE','IT PROFESSIONAL'
 ,'DOCTOR/DENTIST/MEDICAL PROFESSIONAL','LAWYER','TEACHER/INSTRUCTOR/COACH'
 ,'ADMIN/SECRETARIAL']),1,0)
df1['natureofwork:BIN3']=np.where(df1['natureofwork'].isin(['CONSULTANT','INSURANCE AGENT/FINANCIAL ADVISOR'
,'OTHER PROFESSIONAL SERVICES','GOVT EMPLOYEE','ENTERTAINER/ARTIST']),1,0)
df1['natureofwork:BIN4']=np.where(df1['natureofwork'].isin(['OTHER NON PROFESSIONAL SERVICES','SALES MARKETING PROFESSIONAL'
 ,'CALL CENTER AGENT/TELE MARKETER']),1,0)
df1['natureofwork:BIN0']=np.where((df1['natureofwork'].isin(['CASHIER/FOOD SERVER/WAITER','SALES/MARKETING PROFESSIONAL'
,'UTILITY PERSONNEL/HOUSEHOLD HELP','BROKER/DEALER']) | (df1['natureofwork'].isnull())),1,0)

################################# custLoanJourneyProvince ########################################

df1['loanProvince:BIN1']=np.where(df1['custLoanJourneyProvince'].isin(['ABRA','LANAO DEL SUR','SOUTHERN LEYTE','DAVAO OCCIDENTAL'
 ,'ZAMBOANGA DEL NORTE','SARANGANI','CATANDUANES','AGUSAN DEL NORTE'
 ,'DAVAO DE ORO (COMPOSTELA VALLEY)','SURIGAO DEL SUR','BENGUET'
 ,'DAVAO ORIENTAL','MASBATE','EASTERN SAMAR','GUIMARAS','PALAWAN'
 ,'ZAMBOANGA DEL SUR','SOUTH COTABATO','CAGAYAN','PANGASINAN','ALBAY'
 ,'BUKIDNON','SULTAN KUDARAT','MISAMIS OCCIDENTAL','ILOCOS SUR','LA UNION'
 ,'LAGUNA']),1,0)
df1['loanProvince:BIN2']=np.where(df1['custLoanJourneyProvince'].isin(['ANTIQUE','AGUSAN DEL SUR','BATANGAS','CAVITE','BULACAN','CAPIZ','RIZAL']),1,0)
df1['loanProvince:BIN3']=np.where(df1['custLoanJourneyProvince'].isin(['ORIENTAL MINDORO','CAMARINES SUR','KALINGA','BATAAN','SIQUIJOR','CEBU'
 ,'DAVAO DEL NORTE']),1,0)
df1['loanProvince:BIN4']=np.where(df1['custLoanJourneyProvince'].isin(['METRO MANILA','ILOCOS NORTE','SURIGAO DEL NORTE','DAVAO DEL SUR','LEYTE'
 ,'ILOILO','LANAO DEL NORTE','SAMAR (WESTERN SAMAR)']),1,0)
df1['loanProvince:BIN0']=np.where((df1['custLoanJourneyProvince'].isin(['QUEZON','MISAMIS ORIENTAL','NEGROS OCCIDENTAL','PAMPANGA'
 ,'COTABATO (NORTH COTABATO)','AKLAN','NEGROS ORIENTAL','BOHOL','TARLAC'
 ,'CAMARINES NORTE','ZAMBALES','SORSOGON','AURORA','APAYAO'
 ,'MOUNTAIN PROVINCE','NORTHERN SAMAR','OCCIDENTAL MINDORO','ROMBLON'
 ,'BILIRAN','ZAMBOANGA SIBUGAY','MARINDUQUE'])) | (df1['custLoanJourneyProvince'].isnull()),1,0)

################################# purposeDescription ########################################

df1['purposeDescription:BIN1']=np.where(df1['purposeDescription'].isin(['GYM OR FITNESS EQUIPMENT','OVEN, COOKING RANGE, GRILLER'
,'TRAVEL OR VACATION','HELPING FAMILY AND FRIENDS','DEBT REPAYMENT']),1,0)
df1['purposeDescription:BIN2']=np.where(df1['purposeDescription'].isin(['OTHERS','LIVING EXPENSES']),1,0)
df1['purposeDescription:BIN3']=np.where(df1['purposeDescription'].isin(['EDUCATION','HOUSEHOLD APPLIANCES','HOME REPAIR & IMPROVEMENT'
,'ELECTRONICS','WEDDING','HOUSE FURNITURE']),1,0)
df1['purposeDescription:BIN4']=np.where(df1['purposeDescription'].isin(['BED FRAMES OR CRIBS','MOBILE PHONE AND ACCESSORIES'
,'MEDICAL OR OTHER EMERGENCY/HEALTH CARE','WASHING MACHINE, DRYER OR BOTH'
,'DOWN PAYMENT FOR VEHICLE','DINING SET, CHAIRS OR TABLES']),1,0)
df1['purposeDescription:BIN0']=np.where(df1['purposeDescription'].isin(['PC, LAPTOP OR GAME CONSOLES','SALA SET, COUCH, SOFA SET'
,'AIR CONDITIONERS','STEREO SET, SOUNDBAR, AUDIO EQUIPMENT'
,'REFRIGERATORS','TELEVISIONS']),1,0)

################################# doctype1 ########################################

df1['doctype1:BIN1']=np.where(df1['doctype1'].isin(['PHILHEALTH ID','VOTER ID','PRC ID ','PASSPORT']),1,0)
df1['doctype1:BIN2']=np.where(df1['doctype1'].isin(['SOCIAL SECURITY CARD','POSTAL ID']),1,0)
df1['doctype1:BIN3']=np.where(df1['doctype1'].isin(['UMID']),1,0)
df1['doctype1:BIN0']=np.where(df1['doctype1'].isin(['DRIVERS LICENSE','NATIONAL ID']) |(df1['doctype1'].isnull()),1,0)


################################# Gender ########################################

df1['gender:BIN1']=np.where(df1['gender'].isin(['F']),1,0)
df1['gender:BIN0']=np.where(df1['gender'].isin(['M']) |(df1['gender'].isnull()),1,0)



df1.reset_index(inplace=True)




In [ ]:
df2= df1.loc[:,['Credo:1', 'Credo:2', 'Credo:3', 'Credo:4','Credo:5',
       'tsaOpenedMinutes:1', 'tsaOpenedMinutes:2','tsaOpenedMinutes:3', 'tsaOpenedMinutes:4', 
       'industrysubindustry:BIN1', 'industrysubindustry:BIN2','industrysubindustry:BIN3', 'industrysubindustry:BIN4',
       'loanRequestTenure:BIN1','loanRequestTenure:BIN2', 'loanRequestTenure:BIN3','loanRequestTenure:BIN4',
       'monthlyIncome:BIN1', 'monthlyIncome:BIN2', 'monthlyIncome:BIN3','monthlyIncome:BIN4', 
       'natureofwork:BIN1','natureofwork:BIN2', 'natureofwork:BIN3', 'natureofwork:BIN4',
       'loanProvince:BIN1', 'loanProvince:BIN2','loanProvince:BIN3', 'loanProvince:BIN4',
       'purposeDescription:BIN1', 'purposeDescription:BIN2','purposeDescription:BIN3', 'purposeDescription:BIN4',
       'doctype1:BIN1', 'doctype1:BIN2','doctype1:BIN3', 
       'gender:BIN1',
       'DefFSTPD30']]

########################################### ChatGPT Based Working #############################################################

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# Load and preprocess the data
data = df2  # Replace 'your_data.csv' with your actual data file
X = data.drop('DefFSTPD30', axis=1)
y = data['DefFSTPD30']

# Split the data into train, test, and holdout sets
X_train_val, X_holdout, y_train_val, y_holdout = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the train_val set into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train_val, y_train_val, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_holdout = scaler.transform(X_holdout)

# Define the models with their respective hyperparameter grids for tuning
models = {
    'Logistic Regression': (LogisticRegression(), {}),
    'XGBoost': (XGBClassifier(), {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5]}),
    'LightGBM': (LGBMClassifier(), {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5]}),
    'CatBoost': (CatBoostClassifier(), {'iterations': [100, 200, 300], 'depth': [3, 4, 5]}),
    'Random Forest': (RandomForestClassifier(), {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5]}),
    'SVM': (SVC(probability=True), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
    'Linear Discriminant Analysis': (LinearDiscriminantAnalysis(), {}),
    'Quadratic Discriminant Analysis': (QuadraticDiscriminantAnalysis(), {})
}

# Perform hyperparameter tuning and calculate Gini coefficient for train, test, and holdout sets
best_model = None
best_gini = -np.inf

for model_name, (model, param_grid) in models.items():
    # Perform cross-validation to find the best hyperparameters for the train set
    grid_search_train = GridSearchCV(model, param_grid, scoring='roc_auc', cv=5)
    grid_search_train.fit(X_train, y_train)
    best_model = grid_search_train.best_estimator_
    
    # Calculate Gini coefficient for train set
    train_proba = best_model.predict_proba(X_train)[:, 1]
    train_gini = 2 * roc_auc_score(y_train, train_proba) - 1
    
    # Perform cross-validation to find the best hyperparameters for the test set
    grid_search_test = GridSearchCV(model, param_grid, scoring='roc_auc', cv=5)
    grid_search_test.fit(X_test, y_test)
    best_model = grid_search_test.best_estimator_
    
    # Calculate Gini coefficient for test set
    test_proba = best_model.predict_proba(X_test)[:, 1]
    test_gini = 2 * roc_auc_score(y_test, test_proba) - 1
    
    # Perform cross-validation to find the best hyperparameters for the holdout set
    grid_search_holdout = GridSearchCV(model, param_grid, scoring='roc_auc', cv=5)
    grid_search_holdout.fit(X_holdout, y_holdout)
    best_model = grid_search_holdout.best_estimator_
    
    # Calculate Gini coefficient for holdout set
    holdout_proba = best_model.predict_proba(X_holdout)[:, 1]
    holdout_gini = 2 * roc_auc_score(y_holdout, holdout_proba) - 1
    
    print(f'{model_name} Gini Coefficient:')
    print(f'Train Gini: {train_gini}')
    print(f'Test Gini: {test_gini}')
    print(f'Holdout Gini: {holdout_gini}')
    print()
    
    if holdout_gini > best_gini:
        best_gini = holdout_gini

print('Best Model:', best_model)
print('Best Holdout Gini:', best_gini)


KeyboardInterrupt: 